In [1]:
import os
import zipfile
from pyspark.sql import SparkSession

24/12/30 00:20:52 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
spark = SparkSession.builder.appName("RestaurantOrdersPipeline").getOrCreate()

In [14]:
def extract_zip(zip_file_path, extract_to_dir):
    """
    Extract ZIP file to a specified directory.
    """
    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to_dir)
    print(f"Extracted {zip_file_path} to {extract_to_dir}")

In [17]:
def process_restaurant_orders_with_menu(file_path_orders, file_path_menu):
    """
    Process the Restaurant Orders dataset with PySpark and join it with menu items.
    """
    orders_df = spark.read.csv(file_path_orders, header=True, inferSchema=True)
    menu_df = spark.read.csv(file_path_menu, header=True, inferSchema=True)

    print("Orders DataFrame Schema:")
    orders_df.printSchema()

    print("Menu Items DataFrame Schema:")
    menu_df.printSchema()
    menu_df = menu_df.withColumnRenamed("menu_item_id", "item_id")

    # Join the datasets on the item_id column
    joined_df = orders_df.join(menu_df, on="item_id", how="inner")
    print("Joined DataFrame:")
    joined_df.show(truncate=False)

    #Total revenue per date
    revenue_per_date = joined_df.groupBy("order_date").agg((joined_df["price"]).alias("TotalRevenue")).orderBy("order_date")
    print("Total Revenue Per Date:")
    revenue_per_date.show()

    #Top-selling items
    top_items = joined_df.groupBy("item_name").count().orderBy("count", ascending=False)
    print("Top-Selling Items:")
    top_items.show()

    #Revenue by item category
    revenue_by_category = joined_df.groupBy("category").agg((joined_df["price"]).alias("TotalRevenue")
                                                           ).orderBy("TotalRevenue", ascending=False)

    print("Revenue By Item Category:")
    revenue_by_category.show()

    # Save the analysis results
    revenue_per_date.write.csv("output/revenue_per_date.csv", mode="overwrite", header=True)
    top_items.write.csv("output/top_items.csv", mode="overwrite", header=True)
    revenue_by_category.write.csv("output/revenue_by_category.csv", mode="overwrite", header=True)
    print("Analysis results saved to the output directory.")


In [19]:
def main():
    zip_file_path = "data/RestaurantOrders.zip"  
    extract_to_dir = "extracted_files"  

    #Extract ZIP file
    if not os.path.exists(extract_to_dir):
        os.makedirs(extract_to_dir)
    extract_zip(zip_file_path, extract_to_dir)

    #Process each extracted file
    for file_name in os.listdir(extract_to_dir):
        if file_name == 'restaurant_db_data_dictionary.csv':
            return
        file_path = os.path.join(extract_to_dir, file_name)
        process_restaurant_orders(file_path)

    # Stop Spark Session
    spark.stop()

if __name__ == "__main__":
    main()

Extracted data/RestaurantOrders.zip to extracted_files
Loaded DataFrame Schema:
root
 |-- order_details_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- order_date: string (nullable = true)
 |-- order_time: string (nullable = true)
 |-- item_id: string (nullable = true)

+----------------+--------+----------+-----------+-------+
|order_details_id|order_id|order_date|order_time |item_id|
+----------------+--------+----------+-----------+-------+
|1               |1       |1/1/23    |11:38:36 AM|109    |
|2               |2       |1/1/23    |11:57:40 AM|108    |
|3               |2       |1/1/23    |11:57:40 AM|124    |
|4               |2       |1/1/23    |11:57:40 AM|117    |
|5               |2       |1/1/23    |11:57:40 AM|129    |
|6               |2       |1/1/23    |11:57:40 AM|106    |
|7               |3       |1/1/23    |12:12:28 PM|117    |
|8               |3       |1/1/23    |12:12:28 PM|119    |
|9               |4       |1/1/23    |12:16:31 PM|11